# IMPORTING THE LIBRARIES

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


# IMPORTING THE DATASET

In [2]:
dataset=pd.read_csv('diabetic_data.csv')
import pandas as pd

dataset = pd.read_csv("diabetic_data.csv")

drop_cols = [

    "encounter_id", "patient_nbr",

    "race", "weight", "payer_code",

    "discharge_disposition_id",
    "admission_source_id",
    "admission_type_id",


    "max_glu_serum", "A1Cresult",

    "metformin", "repaglinide", "nateglinide", "chlorpropamide",
    "glimepiride", "acetohexamide", "glipizide", "glyburide",
    "tolbutamide", "pioglitazone", "rosiglitazone", "acarbose",
    "miglitol", "troglitazone", "tolazamide", "examide",
    "citoglipton",


    "glipizide-metformin", "glyburide-metformin",
    "metformin-pioglitazone", "metformin-rosiglitazone",
    "glimepiride-pioglitazone"
]

dataset = dataset.drop(columns=drop_cols)



In [3]:

y = dataset.iloc[:, -1]


# CONVERTING DEPENDENT VARIABLE TO BINARY

In [4]:
y = y.apply(lambda x: 1 if x == '<30' else 0)
print(y)

0       0
1       0
2       0
3       0
4       0
       ..
5530    0
5531    0
5532    0
5533    0
5534    0
Name: readmitted, Length: 5535, dtype: int64


# ENCODING CATEGORICAL DATA

# SPLITTING THE DATASET

# TRAINING THE MODEL LOGISTIC REGRESSION

# METRICS

# Catboost

In [5]:
!pip install catboost


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 9.5 MB/s eta 0:00:00


In [6]:
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split

X_cat = dataset.drop(columns=["readmitted"])
y_cat = y


In [7]:
cat_features = [
    "age",
    "gender",
    "medical_specialty",
    "diag_1",
    "diag_2",
    "diag_3",
    "insulin",
    "change",
    "diabetesMed"
]
XC_train, XC_test, yC_train, yC_test = train_test_split(
    X_cat,
    y_cat,
    test_size=0.2,
    random_state=42,
    stratify=y_cat
)


In [11]:
cat_model = CatBoostClassifier(
    iterations=300,
    learning_rate=0.1,
    depth=6,
    eval_metric="AUC",
    random_seed=42,
    verbose=100
)


for col in cat_features:

    XC_train[col] = XC_train[col].fillna('Unknown').astype(str)
    XC_test[col] = XC_test[col].fillna('Unknown').astype(str)

cat_model.fit(
    XC_train,
    yC_train,
    cat_features=cat_features,
    eval_set=(XC_test, yC_test)
)

0:	test: 0.5000000	best: 0.5000000 (0)	total: 84.3ms	remaining: 25.2s
100:	test: 0.6349505	best: 0.6640176 (39)	total: 2.97s	remaining: 5.85s
200:	test: 0.6351086	best: 0.6640176 (39)	total: 6.59s	remaining: 3.25s
299:	test: 0.6307647	best: 0.6640176 (39)	total: 9.9s	remaining: 0us

bestTest = 0.6640176417
bestIteration = 39

Shrink model to first 40 iterations.


In [12]:
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score

yC_pred = cat_model.predict(XC_test)
yC_pred_proba = cat_model.predict_proba(XC_test)[:, 1]

print(confusion_matrix(yC_test, yC_pred))
print(classification_report(yC_test, yC_pred))
print("ROC-AUC:", roc_auc_score(yC_test, yC_pred_proba))


[[985   0]
 [122   0]]
              precision    recall  f1-score   support

           0       0.89      1.00      0.94       985
           1       0.00      0.00      0.00       122

    accuracy                           0.89      1107
   macro avg       0.44      0.50      0.47      1107
weighted avg       0.79      0.89      0.84      1107

ROC-AUC: 0.6640176416742947


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [13]:
cat_model.save_model("readmission_catboost.cbm")
